In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load data
train_df = pd.read_csv('train_labels.csv')
test_df = pd.read_csv('sample_submission.csv')

# Data preprocessing and splitting
train_df['id'] = train_df['id'].apply(lambda x: x + '.tif')
test_df['id'] = test_df['id'].apply(lambda x: x + '.tif')
train_df['label'] = train_df['label'].astype(str)
train_df, validation_df = train_test_split(train_df, stratify=train_df['label'], test_size=0.1, random_state=42)

# Data generators and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, vertical_flip=True, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=train_df, directory='train/', x_col='id', y_col='label', target_size=(96, 96), batch_size=32, class_mode='binary', subset='training')
validation_generator = train_datagen.flow_from_dataframe(dataframe=train_df, directory='train/', x_col='id', y_col='label', target_size=(96, 96), batch_size=32, class_mode='binary', subset='validation')
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df, directory='test/', x_col='id', y_col=None, target_size=(96, 96), batch_size=32, class_mode=None, shuffle=False)

# Build the model
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Evaluate the model
validation_generator.reset()
y_pred = model.predict(validation_generator)
y_pred_binary = (y_pred > 0.5).astype(int)
y_true = validation_generator.classes

accuracy = accuracy_score(y_true, y_pred_binary)
confusion = confusion_matrix(y_true, y_pred_binary)
report = classification_report(y_true, y_pred_binary)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion}')
print(f'Classification Report:\n{report}')

# Make predictions on the test set
test_generator.reset()
test_pred = model.predict(test_generator)
test_df['label'] = test_pred
test_df.to_csv('submission.csv', index=False)
